In [ ]:
%load_ext autoreload
%autoreload 2
    
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import random

from scipy.optimize import curve_fit

from functionsFIMS import getAnalysisNumbers
from runDataClass import runData
from polyaClass import myPolya

In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(1148)
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    binWidth=3
    simData.plotAvalancheSize(binWidth)
    simData.plotAvalancheFits(binWidth)

    thresh = 10
    fitPolya = myPolya(24.701, 0.488)
    efficiency = fitPolya.calcEfficiencyErrs(thresh, 0.671, 0.095)
    print(f'{efficiency[0]:.3f} (+{efficiency[1]:.3f}, {efficiency[2]:.3f})')

In [ ]:
if __name__ == '__main__':
    randInt = random.randint(0, 999)
    simData.plotAvalanche2D(randInt, plotName='Random')
    simData.plotAvalancheSignal(randInt)
    simData.plotSignalvsGain()

In [ ]:
TESTDATA = runData(1172)

In [ ]:
def getLargestRadius(xData, yData):
    """
    TODO
    """
    radius2 = xData**2 + yData**2
    maxRadius = math.sqrt(radius2.max())
    minRadius = math.sqrt(radius2.min())

    return maxRadius, minRadius

    

In [ ]:
def showIonStarts(runData):
    """
    TODO
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]


    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))    
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad, 
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))   
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('Positive Ion Initial Locations')
    ax = fig.add_subplot(111)

    ax.scatter(
        bfIons['Initial x'], bfIons['Initial y'],
        label='BF Ions', c='r', marker='.', alpha=.15
    )
    ax.scatter(
        capIons['Initial x'], capIons['Initial y'],
        label='Captured Ions', c='g', marker='.', alpha=0.15
    )

    ax.add_patch(bfCircle)
    ax.add_patch(capCircle0)
    ax.add_patch(capCircle1)
    
    
    runData._plotAddCellGeometry(ax, 'xy')

    padLength = runData.getRunParameter('Pad Length')
    pitch = runData.getRunParameter('Pitch')
    axLim = pitch/math.sqrt(3)
    ax.set_xlim(-axLim, axLim)
    ax.set_ylim(-axLim, axLim)
    ax.legend()

    return fig

_ = showIonStarts(TESTDATA)

In [ ]:
def showIonHist(runData):
    """
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    posIons = posIons[posIons['Initial z']!=posIons['Initial z'].iloc[0]]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]

    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad,
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad,
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad,
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    nominalFieldBundle = plt.Circle(
            (0, 0), runData.getRunParameter('Field Bundle Radius'), 
            facecolor='none', edgecolor='c', lw=2
        )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('All Ion Initial Locations')
    ax = fig.add_subplot(111)
    '''
    ax.hist2d(
        posIons['Initial x'], posIons['Initial y'],
        bins=101, cmin=1
    )
    '''
    '''
    ax.hist2d(
        bfIons['Initial x'], bfIons['Initial y'],
        bins=101, cmin=1
    )
    '''
    ax.hist2d(
        capIons['Initial x'], capIons['Initial y'],
        bins=101, cmin=1
    )

    #ax.add_patch(bfCircle)
    #ax.add_patch(capCircle0)
    #ax.add_patch(capCircle1)
    ax.add_patch(nominalFieldBundle)
    
    runData._plotAddCellGeometry(ax, 'xy')

    padLength = runData.getRunParameter('Pad Length')
    pitch = runData.getRunParameter('Pitch')
    
    largeLim = pitch/math.sqrt(3)
    axLim = padLength
    ax.set_xlim(-axLim, axLim)
    ax.set_ylim(-axLim, axLim)
    #ax.legend()

    return fig

_ = showIonHist(TESTDATA)

In [ ]:
def getLargestRadius(xData, yData):
    """
    TODO
    """
    radius2 = xData**2 + yData**2
    maxRadius = math.sqrt(radius2.max())
    minRadius = math.sqrt(radius2.min())

    return maxRadius, minRadius

    

In [ ]:
def showIonStarts(runData):
    """
    TODO
    """
    allIons = runData.getDataFrame('ionData')
    posIons = allIons[allIons['Ion Charge']==1]
    
    bfIons = posIons[posIons['Final z']>=1]
    capIons = posIons[posIons['Final z']<1]


    bfMaxRad, bfMinRad = getLargestRadius(np.array(bfIons['Initial x'].values), np.array(bfIons['Initial y'].values))    
    bfCircle = plt.Circle(
        (0, 0), bfMaxRad, 
        facecolor='none', edgecolor='r', ls=':', lw=1, label='Backflow Region'
    )
    
    capMaxRad, capMinRad = getLargestRadius(np.array(capIons['Initial x'].values), np.array(capIons['Initial y'].values))   
    capCircle0 = plt.Circle(
        (0, 0), capMaxRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1, label='Captured Region'
    )
    capCircle1 = plt.Circle(
        (0, 0), capMinRad, 
        facecolor='none', edgecolor='g', ls=':', lw=1
    )

    fig = plt.figure(figsize=(10, 10))
    fig.suptitle('Positive Ion Initial Locations')
    ax = fig.add_subplot(111)

    ax.scatter(
        bfIons['Initial x'], bfIons['Initial y'],
        label='BF Ions', c='r', marker='.'
    )
    ax.scatter(
        capIons['Initial x'], capIons['Initial y'],
        label='Captured Ions', c='g', marker='.'
    )

    ax.add_patch(bfCircle)
    ax.add_patch(capCircle0)
    ax.add_patch(capCircle1)
    
    
    runData._plotAddCellGeometry(ax, 'xy')

    #axLim = runData.getRunParameter('Pad Length')
    #ax.set_xlim(-axLim, axLim)
    #ax.set_ylim(-axLim, axLim)
    ax.legend()

    return fig

_ = showIonStarts(TESTDATA)

In [ ]:
gridPixData = runData(20008)
FIMSData = runData(1173)

In [ ]:
_ = gridPixData.plotAvalancheFits(binWidth=50)
_ = FIMSData.plotAvalancheFits(binWidth=1)

In [ ]:
#efficiencyRuns = np.arange(1200, 1205)
efficiencyRuns = np.arange(1205, 1210)

allRunData = []

params = [
    'Electric Field Ratio',
    'Raw Gain',
    'Trimmed Gain',
    'Raw Efficiency (10e)',
    'Raw Efficiency Error',
    'Efficiency (10e)',
    'Efficiency Error'
]

for inRun in efficiencyRuns:
    inRunData = runData(inRun)

    thisRunData = {'runNumber': inRun}
    
    # Loop over your keys to populate the dictionary
    for inParam in params:
        thisRunData[inParam] = inRunData.getRunParameter(inParam)

    allRunData.append(thisRunData)

simData = pd.DataFrame(allRunData)


fig, axes = plt.subplots(1, 3, figsize=(12, 4))
fig.suptitle('FIMS Simulations')

axes[0].errorbar(
    simData['Electric Field Ratio'], simData['Raw Gain'], yerr=np.sqrt(simData['Raw Gain']),
    label=f'Raw Gain', ls='', marker='x'
)

axes[1].errorbar(
    simData['Electric Field Ratio'], simData['Raw Efficiency (10e)'], yerr=simData['Raw Efficiency Error'],
    label='Raw Efficiency (10e)', ls='', marker='x'
)

axes[2].errorbar(
    simData['Raw Gain'], simData['Raw Efficiency (10e)'], 
    xerr=np.sqrt(simData['Raw Gain']), yerr=simData['Raw Efficiency Error'],
    label='Raw Efficiency (10e)', ls='', marker='x'
)

axes[0].errorbar(
    simData['Electric Field Ratio'], simData['Trimmed Gain'], yerr=np.sqrt(simData['Trimmed Gain']),
    label=f'Trimmed Gain', ls='', marker='x'
)

axes[1].errorbar(
    simData['Electric Field Ratio'], simData['Efficiency (10e)'], yerr=simData['Efficiency Error'],
    label='Efficiency (10e)', ls='', marker='x'
)

axes[2].errorbar(
    simData['Trimmed Gain'], simData['Efficiency (10e)'], 
    xerr=np.sqrt(simData['Raw Gain']), yerr=simData['Efficiency Error'],
    label='Efficiency (10e)', ls='', marker='x'
)



axes[1].axhline(y=.95, label='95% Target', c='g', ls=':')
axes[2].axhline(y=.95, label='95% Target', c='g', ls=':')
axes[0].set_yscale('log')
axes[2].set_xscale('log')


for inax in axes.flat:
    inax.grid()
    inax.legend()

plt.tight_layout()
plt.show()




In [ ]:
def myExpo(x, a, b, c):
    return a*np.exp(b*(x-c))

def mySigmoid(x, k, x0):
    return 1/(1+np.exp(-k*(x-x0)))


fitFields = np.arange(min(simData['Electric Field Ratio'])-5, max(simData['Electric Field Ratio'])+5)

l0 = [1, 0.2, 20]
lopt, lcov = curve_fit(
    myExpo, simData['Electric Field Ratio'], simData['Trimmed Gain'], 
    l0, method='lm', sigma=np.sqrt(simData['Trimmed Gain']), absolute_sigma=True
)
fitExpo = myExpo(fitFields, *lopt)

p0 = [0.5, 40]
popt, pcov = curve_fit(
    mySigmoid, simData['Electric Field Ratio'], simData['Raw Efficiency (10e)'], 
    p0, method='lm', sigma=simData['Raw Efficiency Error'], absolute_sigma=True
)
fitSigmoidRaw = mySigmoid(fitFields, *popt)

popt, pcov = curve_fit(
    mySigmoid, simData['Electric Field Ratio'], simData['Efficiency (10e)'], 
    p0, method='lm', sigma=simData['Efficiency Error'], absolute_sigma=True
)
fitSigmoid = mySigmoid(fitFields, *popt)

print(f'Expo fit parameters: {lopt[0]:.3f}, {lopt[1]:.3f}, {lopt[2]:.3f}')
print(f'Sigmoid fit parameters: {popt[0]:.3f}, {popt[1]:.3f}')

fig = plt.figure(figsize=(10, 4))
fig.suptitle('FIMS Simulations')
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)


ax1.errorbar(
    simData['Electric Field Ratio'], simData['Raw Gain'], 
    yerr=np.sqrt(simData['Raw Gain']), ls='', marker='x', label='Raw Gain'
)
ax1.errorbar(
    simData['Electric Field Ratio'], simData['Trimmed Gain'], 
    yerr=np.sqrt(simData['Trimmed Gain']), ls='', marker='x', label='Trimmed Gain'
)
#ax1.plot(fitFields, fitExpo, label='Exponential Fit', c='r', ls='--')


ax2.errorbar(
    simData['Electric Field Ratio'], simData['Raw Efficiency (10e)'], 
    yerr=simData['Raw Efficiency Error'], ls='', marker='x', label='Raw Efficiency'
)
ax2.errorbar(
    simData['Electric Field Ratio'], simData['Efficiency (10e)'], 
    yerr=simData['Efficiency Error'], ls='', marker='x', label='Efficiency'
)

ax2.plot(fitFields, fitSigmoidRaw, label='Sigmoid Fit (Raw)', c='r', ls=':')
ax2.plot(fitFields, fitSigmoid, label='Sigmoid Fit (Trimmed)', c='r', ls='--')
ax2.axhline(y=.95, label='95% Target', c='g', ls=':')


testX = simData['Electric Field Ratio'].iloc[0:-1]
testY = simData['Trimmed Gain'].iloc[0:-1]
ltest = [1, 0.2, 20]
lopttest, lcovtest = curve_fit(
    myExpo, testX, testY, 
    l0, method='lm', sigma=np.sqrt(testY), absolute_sigma=True
)
testExpo = myExpo(fitFields, *lopttest)
print(f'Test Expo fit parameters: {lopttest[0]:.3f}, {lopttest[1]:.3f}, {lopttest[2]:.3f}')
ax1.plot(fitFields, testExpo, label='Exponential Fit', c='r', ls=':')

axlim = [.9, 1.1e3]

ax1.set_title('Gain')
ax1.set_xlabel('Field Ratio')
ax1.set_ylabel('Trimmed Gain')
ax1.set_yscale('log')
ax1.set_ylim(axlim)
ax1.legend()
ax1.grid()

ax2.set_title('Efficiency')
ax2.set_xlabel('Field Ratio')
ax2.set_ylabel('10-Electron Efficiency')
ax2.legend()
ax2.grid()

plt.tight_layout()
plt.show()
